In [1]:
sc

org.apache.spark.SparkContext@7136e0e0

In [2]:
spark

org.apache.spark.sql.SparkSession@5757f5a4

In [12]:
/*1. Write a function that will put N doubles into a file. The doubles need to be normally distributed
with mean 0 and standard deviation 1. The function should have two arguments: N
and the full name of the file (ie includes path to file location).
import java.io._*/


def doublesInFile(n:Int, filePath:String) : Array[Double] = {
    val r = scala.util.Random

    var doublesArr:Array[Double] = new Array[Double](n)    //instantiates an array of size n
    //print("Size: "+doublesArr.size)

    for(i<-0 until n ) {
      doublesArr(i) = r.nextGaussian()
    }

    //writing to the filePath provided
    val pw = new PrintWriter(filePath)
    val newLine = sys.props("line.separator")              //get the '\n' for scala

    for(i<-doublesArr.indices ){
      pw.write(doublesArr(i).toString+newLine)
    }
    pw.close()
    println("Successful!")

    doublesArr
  }//ebd of def


In [13]:
doublesInFile(50000,"/home/shahyash1993/bigData/assigment3/doublesFile.txt")

Successful!


Array(1.2430155244701695, -1.3183358442110307, 0.47143738676044433, -0.9815809241457173, -1.667834454737898, 0.44355552651587077, -0.49388166209720363, -1.2106055737633865, 1.107426809790712, 0.5985527826794214, 0.3625095620722051, 0.004776243716141819, -0.3842201375394312, 0.7313831451629214, -0.40982748007376607, 1.1157199716652988, 1.3492327583712227, 0.9462376274671586, -0.4439225291400797, 0.1843440247568084, 1.4972259017099598, -0.5007040375924772, -1.102244169309396, -0.257893069548835, -0.6425800010028886, 0.05817705958574153, 0.4218052277262012, -1.290404510837337, -0.2954986126142325, -0.4433706321646232, 0.6658550236796856, 1.2296409030157955, -1.2540185915203448, -0.3563730829533177, 0.480657164972608, -0.06307687175968754, -0.057141479776724936, -0.01...

In [26]:
 /*2. Read the file created in #2 into an RDD and compute the mean and standard deviation of
the doubles in the file. Work on the RDD, that is do not convert the RDD to a DataFrame or
Dataset.. You are to use Spark code to compute the values as we want this to run on a
cluster using multiple machines. So the pure Scala code you used in assignment will not
work.*/
 
 
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
 
def readRDDMeanSD(): Tuple2[Double,Double]  = {
      val filePath = "/home/shahyash1993/bigData/assigment3/doublesFile.txt"
      val conf = new SparkConf().setAppName("readRDDMeanSD").setMaster("local[*]")

      val minPartitions = 2
      val doublesData:RDD[Double] = sc.textFile(filePath, minPartitions).map(_.toDouble)   

      val mean:Double = doublesData.stats().mean      //using stat to find mean and the standard deviation
      val SD:Double = doublesData.stats().stdev
      val popSD:Double = doublesData.stats().popStdev

      println("Mean is: "+mean + " & SD is: "+SD+ " & popSD is: "+popSD)
      
      Tuple2(mean,SD)
}

In [27]:
readRDDMeanSD()

Mean is: -0.0029387081644154257 & SD is: 1.004153011757705 & popSD is: 1.004153011757705


(-0.0029387081644154257,1.004153011757705)

In [32]:
/*3. Repeat #3 but using a DataFrame instead of RDD. Here work on the DataFrame not an RDD*/


import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}
import org.apache.spark.sql.functions._

def usingDF():Unit = {
    val filePath = "/home/shahyash1993/bigData/assigment3/doublesFile.txt"
    val conf = new SparkConf().setAppName("usingDF").setMaster("local[*]")
    
    val sqlContext: SQLContext = spark.sqlContext

    val minPartitions = 2
    val doublesRDD:RDD[Double] = sc.textFile(filePath, minPartitions).map(_.toDouble)   //gets the whole RDD elements in Double

    import sqlContext.implicits._
    val doublesDF = doublesRDD.toDF("Numbers")

    println("Mean,SD,popSD of the newly generated DataFrame is: ")
    
    doublesDF.select(avg($"Numbers")).show()          
    doublesDF.select(stddev($"Numbers")).show()
    doublesDF.select(stddev_pop($"Numbers")).show()
  }


In [33]:
usingDF()

Mean,SD,popSD of the newly generated DataFrame is: 
+--------------------+
|        avg(Numbers)|
+--------------------+
|-0.00293870816441...|
+--------------------+

+--------------------+
|stddev_samp(Numbers)|
+--------------------+
|   1.004163053438448|
+--------------------+

+-------------------+
|stddev_pop(Numbers)|
+-------------------+
| 1.0041530117577049|
+-------------------+



In [35]:
/*4. Using a DataFrame create a random sample of about 100 elements of the file created in #2
and compute the mean of the sample.*/


import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{SQLContext, SparkSession}
import org.apache.spark.sql.functions.{avg, stddev, stddev_pop}

  def randomSampleMean(): Unit = {

    val filePath = "/home/shahyash1993/bigData/assigment3/doublesFile.txt"
    val conf = new SparkConf().setAppName("randomSampleMean").setMaster("local[*]")
    val sqlContext: SQLContext = spark.sqlContext

    val doublesDF = spark.read.textFile(filePath)
    //val fraction:Float = 100/50000
    val sampleFromDF= doublesDF.sample(true, 0.002) //0.002 is taken as it is 100/50000
    sampleFromDF.select(mean(sampleFromDF("value"))).show()    
  }//end of def


In [36]:
randomSampleMean()

+-------------------+
|         avg(value)|
+-------------------+
|0.13197156383042807|
+-------------------+



In [48]:
/*5. Create a file of 100 normally distributed doubles. Read the doubles from the file into an
RDD. Using the RDD create a sliding window of size 20 and compute the mean of each
window.*/


import java.io.PrintWriter
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{SQLContext, SparkSession}

def slidingMean(): Unit = {
    val n:Int = 100
    val slidingSize:Int = 20
    val slidingStep:Int = 1

    val path:String = "/home/shahyash1993/bigData/assigment3/doublesFile.txt"
    val r = scala.util.Random
    val conf = new SparkConf().setAppName("Simple Application").setMaster("local[*]")
    val sqlContext: SQLContext = spark.sqlContext

    var doublesArr:Array[Double] = new Array[Double](n)
    //print("Size: "+doublesArr.size)

    for(i<-0 until n ) {
      doublesArr(i) = r.nextGaussian();
    }

    //write to the file
    val printWriter = new PrintWriter(path)
    val newLine = sys.props("line.separator")

    for(i<-0 until doublesArr.length ){
      printWriter.write(doublesArr(i).toString+newLine)
    }
    printWriter.close()

    val minPartitions = 2
    val doublesData:RDD[Double] = sc.textFile(path, minPartitions).map(_.toDouble)

    //find the mean using sliding and map function
    var slidingIterator:Iterator[Double] =  doublesData.collect().sliding(slidingSize,slidingStep).map(x=>x.sum/x.length)

    slidingIterator.foreach((d: Double) => println(d))
  }


In [49]:
slidingMean

0.10436077218370517
0.18394443594753082
0.16473827177191153
0.20710484721936456
0.24147915346032747
0.3998787845278735
0.36788679242480304
0.38869945484381424
0.34519815327614173
0.41895631426490276
0.2800410744192085
0.14831839120460172
0.08666837877933874
-0.048129454341718024
-0.012734167506833993
-0.007644974040424128
0.03589642562373713
-0.03058590176773219
-0.09045392157258517
0.05064692950409153
0.02987036522942607
-0.08109986262683
-0.01566101707374563
-0.029023508376274442
0.03590138971687899
-0.06928944010805489
-0.048179870903485134
-0.09085911434757891
-0.09542432775330627
-0.25253484311374563
-0.1496502871854543
-0.05127417986501544
0.004532394633119179
0.04114923485571435
-0.03342798757690138
-0.05293028301332885
0.023436073591015905
0.010593935581629077
0.05966109739567634
-0.01914384168750461
0.011995042955986945
0.0450098059087092
-0.01745343338353842
0.04374367456759183
-0.035355246720924875
-0.014197875934041499
-0.027116127677916768
-0.06703348973072934
-0.183862248

In [39]:
/*6. The file “multiple-sites.tsv” contains two columns: site and dwell-time. Using Spark compute
the average dwell time for each site.*/

import org.apache.spark.SparkConf
import org.apache.spark.sql.{SQLContext, SparkSession}

def averageDwellTime(): Unit = {
    val conf = new SparkConf()
    conf.setAppName("Word Count")
    conf.setMaster("local[2]")

    val sqlContext: SQLContext = spark.sqlContext
    import sqlContext.implicits._
    val multipleSitesDF = spark.read.format("csv")
        .option("delimiter", "\t")
        .option("header",true)
        .load("multiple-sites.tsv")

    val exprs = multipleSitesDF.columns.map(_ -> "mean").toMap

    println("average Dwell-time is: ")
    var avg = multipleSitesDF.groupBy($"site")
                .agg(exprs)
                .select("site","avg(dwell-time)")
                .show()

    //println("Schema: "+multipleSitesDF.printSchema())
  }

In [40]:
averageDwellTime()

average Dwell-time is: 
+----+------------------+
|site|   avg(dwell-time)|
+----+------------------+
|   7|123.36734693877551|
|  15|119.34782608695652|
|  11| 96.98214285714286|
|   3| 97.47916666666667|
|   8| 94.34693877551021|
|  16| 86.74418604651163|
|   0| 79.85106382978724|
|   5|102.33333333333333|
|  18| 94.81481481481481|
|  17|  77.8913043478261|
|   6|144.19298245614036|
|  19| 89.28070175438596|
|   9|             89.82|
|   1|             106.0|
|  10|129.95238095238096|
|   4| 94.33333333333333|
|  12| 80.95081967213115|
|  13| 90.73770491803279|
|  14| 74.76470588235294|
|   2| 88.22916666666667|
+----+------------------+



In [44]:
/*7. The file “multiple-sites.tsv” contains two columns: date and dwell-time. Using Spark compute
the following:
1. The average dwell time each hour
2. The average dwell time per day of week
3. The average dwell time on week-days (Monday - Friday)
4. Average dwell time on the weekend.*/


def averageDwellTimePro(): Unit = {

      val conf = new SparkConf()
      conf.setAppName("Word Count")
      conf.setMaster("local[2]")

      val sqlContext: SQLContext = spark.sqlContext

      val dwellTimesDF = sqlContext.read.format("csv")
        .option("delimiter", "\t")
        .option("header","true")
        .load("dwell-times.tsv")
        .toDF("date","dwell-time")


      import org.apache.spark.sql.functions.{expr, col, column}
      import org.apache.spark.sql.functions._

//1. The average dwell time each hour
      println("1. The average dwell time each hour: ")
      
      val withHour=dwellTimesDF.withColumn("hour", hour(col("date")))
        .groupBy("hour")
        .agg(mean("dwell-time"))
        .sort(col("hour"))
        .show()

//2. The average dwell time per day of week
      println("2. The average dwell time per day of week: ")
      
      val withPerDayOfWeek=dwellTimesDF.withColumn("dayOfWeek", date_format(col("date"),"EEEE"))
        .groupBy("dayOfWeek")
        .agg(mean("dwell-time"))
        .show()

//3. The average dwell time on week-days (Monday - Friday)
      println("3. The average dwell time on week-days (Monday - Friday): ")
      
      val withWeekDays=dwellTimesDF.withColumn("weekDays", date_format(col("date"),"EEEE"))
        .groupBy("weekDays")
        .agg(mean("dwell-time"))
        .where(col("weekDays")!=="Saturday")
        .where(col("weekDays")!=="Sunday")
        .show()

//4. Average dwell time on the weekend.*/
      println("4. Average dwell time on the weekend: ")
      
      val withWeekEnd=dwellTimesDF.withColumn("weekEnd", date_format(col("date"),"EEEE"))
        .groupBy("weekEnd")
        .agg(mean("dwell-time"))
        .filter(col("weekEnd")==="Saturday" || col("weekEnd")==="Sunday" )
        .show()
    }

In [45]:
averageDwellTimePro()

The average dwell time each hour: 
+----+-----------------+                                                        
|hour|  avg(dwell-time)|
+----+-----------------+
|   0|  94.708670095518|
|   1|92.20954287620954|
|   2|96.85937970490816|
|   3|92.57110924839341|
|   4|91.33086825527253|
|   5|96.14621040723982|
|   6|92.17173727608757|
|   7|94.68557487038731|
|   8|92.35551839464883|
|   9|90.15113156885309|
|  10|92.52882433045846|
|  11|92.37820848611838|
|  12|91.21507890122736|
|  13|92.82834185536888|
|  14|95.47408666100254|
|  15|90.56982193064667|
|  16| 93.1575817641229|
|  17|91.66231155778894|
|  18|93.13833634719711|
|  19| 88.4258114374034|
+----+-----------------+
only showing top 20 rows

The average dwell time per day of week: 
+---------+------------------+
|dayOfWeek|   avg(dwell-time)|
+---------+------------------+
|Wednesday| 91.57283771155643|
|  Tuesday| 89.73922367574522|
|   Friday| 90.44266729389628|
| Thursday|  91.4947995556902|
| Saturday|116.8825301204

In [50]:
/*8. Do the average dwell times computed in #7 indicate any difference in users behavior?*/

/*
1. Average behaviour each hour does not specify any specific difference in users behaviour

2. Average behaviour each day of week specifies that the users' average dwell-time is highest on Sunday and lowest on Tuesdays

3. Average behaviour each week-days vs. weekend specifies that the users' average dwell-time is higher on weekend than on weekdays.
*/

sc

org.apache.spark.SparkContext@7136e0e0